In [ ]:
import cv2
from dataset import valid_transform
import glob
import torchvision.models as tm
import torch

model = tm.densenet121(weights=tm.DenseNet121_Weights.DEFAULT)
model.classifier = torch.nn.Linear(model.classifier.in_features, 2)
ckpt = torch.load("/home/songwei/Template/checkpoints/baseline/last.ckpt")

for k, v in list(ckpt["state_dict"].items()):
    if "model." in k:
        ckpt["state_dict"][k[6:]] = ckpt["state_dict"].pop(k)

model.load_state_dict(ckpt["state_dict"])
model.cuda(1)

In [ ]:
test_path = "/home/songwei/Template/data/test"
test_list = glob.glob(test_path + "/*.png")
test_list = sorted(test_list)
predictions = []

for path in test_list:
    print(path)
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    img = valid_transform(image=img)["image"]
    img = img.unsqueeze(0).cuda(1)
    model.eval()
    with torch.no_grad():
        pred = (
            model(img)
            + model(img + 0.1)
            + model(img - 0.1)
            + model(img + 0.2)
            + model(img - 0.2)
            + model(img * 1.1)
            + model(img * 0.9)
        )
    predictions.append(pred.argmax().item())